# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders=pd.read_csv('Orders.csv')
pd.set_option('display.max_columns', None)
display(orders.head())
orders.shape

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


(397924, 14)

In [18]:
orders['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Greece', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Israel', 'Saudi Arabia', 'Czech Republic',
       'Canada', 'Unspecified', 'Brazil', 'USA', 'European Community',
       'Bahrain', 'Malta', 'RSA'], dtype=object)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# Pseudo Code - Step 1:
# I have to GROUP BY the CustomerID and SUM the amount_spent for each customer.
# IN SQL is: SELECT CustomerID, SUM(amount_spent) FROM orders GROUP BY Customerid;
# in Pandas: FROM Orders GROUP BY CustomerID SELECT CustomerID, SUM(amount_spent)  
best_clients=orders.groupby('CustomerID').agg({'amount_spent': 'sum'})

In [4]:
orders['amount_spent'].describe().T

count    397924.000000
mean         22.394749
std         309.055588
min           0.000000
25%           4.680000
50%          11.800000
75%          19.800000
max      168469.600000
Name: amount_spent, dtype: float64

In [5]:
#Pseudo code - Step 2:
#First I have to filter, order, sort the customer into compartments, bins. Binning it is.
binnames = ["VIP", "Preferred Customers", "Moderate", "Poor Spender", "Useless"]
bins = pd.cut(orders['amount_spent'],5, labels=binnames)
bins
orders['BinnedExpenditure'] = pd.cut(orders['amount_spent'], 5, labels = binnames)
bins.value_counts()


VIP                    397921
Preferred Customers         1
Moderate                    1
Useless                     1
Poor Spender                0
Name: amount_spent, dtype: int64

In [6]:
# I need to transform this result into quantilles then.
vip = np.quantile(best_clients, 0.95)
preferred_min = np.quantile(best_clients, 0.75)
preferred_max = np.quantile(best_clients, 0.95)

In [7]:
# Pseudo code - Step 3 
# I need to create two groups of customers based on their total expenses, VIP and Preferred Customer.
vip = best_clients[best_clients > vip]
preferred_customers = best_clients[(best_clients >= preferred_min) & (best_clients <= preferred_max)]

In [8]:
display(vip.head())
display(vip.shape)
display(preferred_customers.head())
preferred_customers.shape

,amount_spent
CustomerID,
12346,77183.6
12347,NaN
12348,NaN
12349,NaN
12350,NaN


(4339, 1)

,amount_spent
CustomerID,
12346,NaN
12347,4310.00
12348,1797.24
12349,1757.55
12350,NaN


(4339, 1)

In [9]:
display(vip.isna().sum())
preferred_customers.isna().sum()

amount_spent    4122
dtype: int64

amount_spent    3471
dtype: int64

#### Finally, we clean the data from the nulls.

In [10]:
# Drop null values from vip_customers
vip.dropna(inplace=True)

# Drop null values from preferred_customers
preferred_customers.dropna(inplace=True)


In [11]:
display(vip.head())
display(vip.shape)
display(preferred_customers.head())
preferred_customers.shape

,amount_spent
CustomerID,
12346,77183.60
12357,6207.67
12359,6372.58
12409,11072.67
12415,124914.53


(217, 1)

,amount_spent
CustomerID,
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
12356,2811.43


(868, 1)

# Conclusion First Steps: From the total 4339 unique customers, 217 are VIP and 868 are Preferred.

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# Pseudo Code:
# STEP 1 -From the 217 customerIDs that are VIP, how do I LOCATE their country
# STEP 2 - How do I JOIN and COUNT that by the COUNTRY
# STEP 3 - Order by amount spent by country

In [19]:
# Calculating the total expenses for each customer
total_expenses = orders.groupby('CustomerID')['amount_spent'].sum()

# Again calling the VIP quantille
vip_threshold = np.quantile(total_expenses, 0.95)

# Filtering the 'total_expenses' Series to only include VIP customers
vip_customers_expenses = total_expenses[total_expenses >= vip_threshold]

# Extract the 'CustomerID', 'Country', and 'amount_spent' columns from the 'orders' DataFrame for VIP customers
vip_customers = orders.loc[orders['CustomerID'].isin(vip_customers_expenses.index), ['CustomerID', 'Country', 'amount_spent']]

# Group the VIP customers by country and calculate the total sales for each country
vip_by_country = vip_customers.groupby('Country')['amount_spent'].sum().reset_index()

# Sort the DataFrame by 'amount_spent' in descending order
vip_by_country = vip_by_country.sort_values('amount_spent', ascending=False)

#Printing the answer
print(vip_by_country.iloc[0]['Country'])


United Kingdom


In [15]:
vip_by_country.head()


,Country,amount_spent
17,United Kingdom,3417798.95
10,Netherlands,280206.02
5,EIRE,261204.69
0,Australia,130429.20
7,France,106383.09


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
# Pseudo code:
# Same logic for VIP but now I to JOIN the 'vip_by_country' in the last step with the 'customers_by_country'  
# ON 'Country'. 
# This combines the number of VIP and Preferred customers in each country.


In [16]:
# Calculating the total expenses for each customer
total_expenses = orders.groupby('CustomerID')['amount_spent'].sum()

# Calculating the VIP threshold as the 95th percentile of total expenses
vip_threshold = np.quantile(total_expenses, 0.95)

# Calculating the preferred threshold as the 75th percentile of total expenses
preferred_threshold = np.quantile(total_expenses, 0.75)

# Filter the 'total_expenses' Series to only include VIP and Preferred customers
vip_customers_expenses = total_expenses[total_expenses >= vip_threshold]
preferred_customers_expenses = total_expenses[(total_expenses >= preferred_threshold) & (total_expenses < vip_threshold)]

# Extract the 'CustomerID' and 'Country' columns from orders for VIP and Preferred customers
vip_customers = orders.loc[orders['CustomerID'].isin(vip_customers_expenses.index), ['CustomerID', 'Country']]
preferred_customers = orders.loc[orders['CustomerID'].isin(preferred_customers_expenses.index), ['CustomerID', 'Country']]

# Combine VIP and Preferred customers into a single DataFrame
vip_and_preferred_customers = pd.concat([vip_customers, preferred_customers])

# Group the customers by country and calculate the number of customers in each country
customers_by_country = vip_and_preferred_customers.groupby('Country')['CustomerID'].nunique().reset_index()

# Sort the DataFrame by the number of customers in descending order
customers_by_country = customers_by_country.sort_values('CustomerID', ascending=False)

# Print the country with the most VIP+Preferred customers combined
print(customers_by_country.iloc[0]['Country'])


United Kingdom


In [17]:
customers_by_country.head()

,Country,CustomerID
26,United Kingdom,932
10,Germany,39
9,France,29
2,Belgium,12
25,Switzerland,9
